In [76]:
import requests
from pathlib import Path
import json
import shutil
import pandas as pd
from tqdm import tqdm
import plotly.express as px

In [60]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100


In [61]:
pd.options.plotting.backend = "plotly"

In [62]:
OUTPUTS = Path("outputs")

In [79]:
loans = []
for file in tqdm(OUTPUTS.glob("*.json")):
    data = json.loads(file.read_text())
    if data["status"] != "OK":
        continue
    loans += json.loads(file.read_text())["loans"]

2165it [00:00, 3349.03it/s]


In [80]:
df = pd.DataFrame(loans)

In [81]:
df

,amount,borrower_rating,company,date,id,image_url,interest_rate,investing,investing_amount,investing_debt_amount,loan_isin,loan_name,loan_order,loan_rating,payments,preview_small_url,preview_url,progress,rating,status,term
0,500000.00,BBB+,"ООО ""АГРООПТ""",2020-09-28T00:00:00+03:00,856,/media/images/ac/7a/ac7a4415-3b39-4d54-b9a0-00...,0.295,-,0.0,0.00,JL0000000856,АГРООПТ-В01,1,C,"[{'date': '2020-10-12T00:00:00+03:00', 'fine':...",/media/images/ac/7a/ac7a4415-3b39-4d54-b9a0-00...,/media/images/ac/7a/ac7a4415-3b39-4d54-b9a0-00...,"[1.0, 0, 0]",C,closed,186
1,1500000.00,BBB+,"ООО ""АГРООПТ""",2021-04-08T11:21:58.454711+03:00,1201,/media/images/ac/7a/ac7a4415-3b39-4d54-b9a0-00...,0.185,-,0.0,0.00,JL0000001201,АГРООПТ-В02,2,BBB+,"[{'date': '2021-05-08T00:00:00+03:00', 'fine':...",/media/images/ac/7a/ac7a4415-3b39-4d54-b9a0-00...,/media/images/ac/7a/ac7a4415-3b39-4d54-b9a0-00...,"[1.0, 0, 0]",BBB+,closed,365
2,898880.00,B,"ООО ""УРАЛ СТРОЙ МОНТАЖ""",2020-09-04T00:00:00+03:00,805,/media/images/84/6f/846f7dd8-a305-4dba-b01a-bc...,0.235,-,0.0,0.00,JL0000000805,УРАЛСТРОЙМОН-В01,1,B,"[{'date': '2020-09-18T00:00:00+03:00', 'fine':...",/media/images/84/6f/846f7dd8-a305-4dba-b01a-bc...,/media/images/84/6f/846f7dd8-a305-4dba-b01a-bc...,"[1.0, 0, 0]",B,closed,217
3,50000.00,None,"ООО НПО ""АЛЬФА КЕМИКАЛ""",2020-02-05T03:00:00+03:00,594,/media/images/63df1d86-a417-4b6f-9819-fa2b4d99...,0.250,-,0.0,0.00,JL0000000411,АЛЬФАКЕМИКАЛ-В02,2,B+,"[{'date': '2020-02-19T00:00:00+03:00', 'fine':...",/media/images/63df1d86-a417-4b6f-9819-fa2b4d99...,/media/images/63df1d86-a417-4b6f-9819-fa2b4d99...,"[0.8386895999999999, 0, 0.1613104000000001]",B+,default,182
4,950000.00,None,"ООО НПО ""АЛЬФА КЕМИКАЛ""",2020-02-05T03:00:00+03:00,595,/media/images/63df1d86-a417-4b6f-9819-fa2b4d99...,0.250,-,0.0,0.00,JL0000000411,АЛЬФАКЕМИКАЛ-В03,3,B+,"[{'date': '2020-02-19T00:00:00+03:00', 'fine':...",/media/images/63df1d86-a417-4b6f-9819-fa2b4d99...,/media/images/63df1d86-a417-4b6f-9819-fa2b4d99...,"[0.38349631578947363, 0, 0.6165036842105264]",B+,default,182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3532,36200.00,CC,ИП Романовский Раймонд Леонидович,2022-01-28T16:34:15+03:00,13848,/media/images/9e966bfb-7bf8-4af3-b300-cd0baa5e...,0.200,-,0.0,0.00,JL0000002237,РомановскийР-В12,12,CC,"[{'date': '2022-02-28T00:00:00+03:00', 'fine':...",/media/images/9e966bfb-7bf8-4af3-b300-cd0baa5e...,/media/images/9e966bfb-7bf8-4af3-b300-cd0baa5e...,"[-0.04973784530386749, 0, 1.0497378453038675]",CC,default,1098
3533,5855200.00,CC,ИП Романовский Раймонд Леонидович,2022-01-28T16:34:15+03:00,13849,/media/images/9e966bfb-7bf8-4af3-b300-cd0baa5e...,0.200,-,0.0,0.00,JL0000002237,РомановскийР-В13,13,CC,"[{'date': '2022-02-28T00:00:00+03:00', 'fine':...",/media/images/9e966bfb-7bf8-4af3-b300-cd0baa5e...,/media/images/9e966bfb-7bf8-4af3-b300-cd0baa5e...,"[0.05939465090859419, 0, 0.9406053490914058]",CC,default,1407
3534,500000.00,CCC,ИП Хатунцев Максим Владимирович,2021-10-12T01:18:05.801633+03:00,1754,/media/images/96/f9/96f98f03-50b2-43d3-9811-f2...,0.240,-,0.0,0.00,JL0000001754,ХатунцевМВ-В01,1,CC+,"[{'date': '2021-11-12T00:00:00+03:00', 'fine':...",/media/images/96/f9/96f98f03-50b2-43d3-9811-f2...,/media/images/96/f9/96f98f03-50b2-43d3-9811-f2...,"[1.0, 0, 0]",CC+,closed,360
3535,1500000.00,CCC,ИП Хатунцев Максим Владимирович,2022-01-26T09:52:31.345146+03:00,2147,/media/images/96/f9/96f98f03-50b2-43d3-9811-f2...,0.240,"#933772, #933607, #933552",0.0,2754.25,JL0000002147,ХатунцевМВ-В02,2,CCC,"[{'date': '2022-02-26T00:00:00+03:00', 'fine':...",/media/images/96/f9/96f98f03-50b2-43d3-9811-f2...,/media/images/96/f9/96f98f03-50b2-43d3-9811-f2...,"[0.27105731999999993, 0, 0.7289426800000001]",CCC,default,516


In [83]:
percentage_df

,rating,status
0,,100.000000
1,A,4.761905
2,A+,0.000000
3,AA,4.210526
4,AA+,0.000000
5,AAA,0.000000
6,AAA+,0.000000
7,B,9.893048
8,B+,10.554090
9,BB,11.228070


In [87]:
percentage_df = df.groupby('rating')['status'].apply(lambda x: (x == 'default').mean() * 100).reset_index()

# Sort by rating
percentage_df = percentage_df.sort_values(by='rating')

# Plot
percentage_df = percentage_df[percentage_df.rating!=""]
fig = px.bar(percentage_df, x='rating', y='status', labels={'status': 'Percentage of Default'})
fig.show()